In [ ]:
# %%

import torch
import einops
import sys

name_to_model_map = {
    "qwen-1.5b": ["Qwen/Qwen2.5-Math-1.5B", "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"],
    "qwen-7b": ["Qwen/Qwen2.5-Math-7B", "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"],
    "qwen-14b": ["Qwen/Qwen2.5-14B", "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"],
}

target_name = "contrastive"
random_ablate = False
amplify = False

target_token_map = {
    "wait" : [" Wait"],
    "deductive": [" Therefore", " Thus"],
    "alternative": [" Alternatively"],
    "contrastive": [" However", " But"]
}

target_token_count = {
    "wait": 3605,
    "deductive": 1275,
    "alternative": 1507,
    "contrastive": 2357,
}

weight_path = "../checkpoints/version_0/qwen-1.5b_6.pt"
weights = torch.load(weight_path)

base_dec = weights["W_dec"][:, 0, :]
reasoning_dec = weights["W_dec"][:, 1, :]
base_norms = torch.norm(base_dec, p=1, dim=1)
reasoning_norms = torch.norm(reasoning_dec, p=1, dim=1)

relative_norms = reasoning_norms / base_norms
normalized_relative_norms = relative_norms / (1 + relative_norms)

/tmp/ipykernel_4101589/1902145077.py:32: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



In [7]:
new_filename = "../results/" + weight_path[3:-3].replace("/","__") + f"_{target_name}.json"

import json
with open(new_filename, 'r') as f:
    data = json.load(f)
    
ablate_features = []

all_active_features = []
for key in data.keys():
    if data[key] > target_token_count[target_name] * 0.6:
        all_active_features.append(int(key))
        rel_norm = reasoning_norms[int(key)] / base_norms[int(key)]
        normalized_relative_norms = rel_norm / (1 + rel_norm)
        if normalized_relative_norms > 0.9:
            ablate_features.append(int(key))

print(ablate_features)

ablate_features = [2,3,4,5]

import random
random.seed(49)
random_ablate_features = random.sample(all_active_features, len(ablate_features))
print(random_ablate_features)

FileNotFoundError: [Errno 2] No such file or directory: '../results/checkpoints__version_0__qwen-1.5b_6_deductive.json'

In [4]:
import sys
sys.path.append("../")

from crosscoder_diff.utils import load_open_reasoning_tokens
all_tokens = load_open_reasoning_tokens()


# %%
from transformer_lens import HookedTransformer, ActivationCache

import os
os.environ['HF_HOME'] = '/om2/user/dbaek/.cache/'

device = "cuda" if torch.cuda.is_available() else "cpu"

save_name = "qwen-1.5b"

name_to_model_map = {
    "qwen-1.5b": ["Qwen/Qwen2.5-Math-1.5B", "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"],
    "qwen-7b": ["Qwen/Qwen2.5-Math-7B", "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"],
    "qwen-14b": ["Qwen/Qwen2.5-14B", "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"],
}


from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(name_to_model_map[save_name][0])


base_model = HookedTransformer.from_pretrained(
    name_to_model_map[save_name][0],
    device=device, 
)

chat_model = HookedTransformer.from_pretrained(
    name_to_model_map[save_name][1],
    device=device,
)
hook_point = f"blocks.{base_model.cfg.n_layers // 2}.hook_resid_pre"

Loading reasoning tokens from disk


/weka/scratch/weka/tegmark/dbaek/deepseek-diff/src/../crosscoder_diff/utils.py:244: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



Loaded pretrained model Qwen/Qwen2.5-Math-1.5B into HookedTransformer


Loaded pretrained model deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B into HookedTransformer


In [14]:
wait_token_ids = set()
for target_token in target_token_map[target_name]:
    token_ids = tokenizer.encode(target_token, add_special_tokens=False)
    print(target_token, token_ids)
    wait_token_ids.update(token_ids)
print("Wait token IDs:", wait_token_ids)

However [11209]
But [3983]
Wait token IDs: {11209, 3983}


In [6]:
amplify_scale = 2.0               # multiply these features by 2.0

# For quick lookup
ablate_features = set(ablate_features)

# Wrap it in a small closure so we can pass "chat_acts" easily
def make_hook_fn(act_A, act_B, pos, model_idx):
    act_all = torch.stack([act_A, act_B], dim=1)
    act_all = act_all[:, :, :pos, :]
    new_act_all = act_all.clone()

    act_all = act_all[:, :, 1:, :] # Discard BOS
    
    # Rearrange so that each token becomes an individual “batch” element.
    # New shape: [ (batch * (seq_len-1)), n_models, d_model ]
    act_all = einops.rearrange(act_all, "batch n_models seq_len d_model -> (batch seq_len) n_models d_model")
    act_all = act_all.to(device)
    act_all = act_all.to(weights["W_enc"].dtype)
    
    # -------- Encode --------
    # x_enc: shape [d_hidden]
    # (batch dimension is "1" here, effectively)
    x_enc = einops.einsum(
        act_all,  # shape [1, n_models, d_model]
        weights["W_enc"],                # shape [n_models, d_model, d_hidden]
        "batch n_models d_model, n_models d_model d_hidden -> batch d_hidden"
    )  # shape [d_hidden]
    x_enc = x_enc + weights["b_enc"]
    print(x_enc.shape)
    
    # -------- Intervene (ablate/amplify) --------
    # Option 1: Zero out some features
    if random_ablate:
        if amplify:
            x_enc[list(random_ablate_features)] *= amplify_scale
        else:
            x_enc[list(random_ablate_features)] = 0.0
    else:
        if amplify:
            x_enc[list(ablate_features)] *= amplify_scale
        else:
            x_enc[list(ablate_features)] = 0.0
    # Option 2: Multiply some features
#    x_enc[list(amplify_features)] *= amplify_scale

    # -------- Decode --------
    # x_dec: shape [n_models, d_model]
    x_dec = einops.einsum(
        x_enc,  # shape [1, d_hidden]
        weights["W_dec"],               # shape [d_hidden, n_models, d_model]
        "b dh, dh nm dm -> b nm dm"
    )  # shape [n_models, d_model]
    x_dec = x_dec + weights["b_dec"]

    recon_act = einops.rearrange(x_dec, "seq_len n_models d_model -> 1 n_models seq_len d_model")
    new_act_all[:, :, 1:, :] = recon_act

    def hook_fn(value, hook):
        return new_act_all[:, model_idx, :, :]
    return hook_fn


total_wait_tokens = 0

orig_base_logit_list = []
orig_chat_logit_list = []
new_base_logit_list = []
new_chat_logit_list = []
for idx, tokens in enumerate(all_tokens[:5]):
    torch.cuda.empty_cache()
    # Convert tokens to a Python list for easy comparison.
    tokens_list = tokens.tolist()
    
    # Find positions in the original sequence where the token is one of our wait tokens.
    wait_positions = [i for i, token_id in enumerate(tokens_list) if token_id in wait_token_ids]
    
    if not wait_positions:
        continue  # Skip sequences with no wait token.
    
    total_wait_tokens += len(wait_positions)
    
    # Ensure tokens has a batch dimension.
    if tokens.ndim == 1:
        tokens = tokens.unsqueeze(0)  # Now shape [1, seq_length]

    with torch.no_grad():
        # Run the base model to get the original logits.
        original_logits = base_model(tokens)

    with torch.no_grad():
        # Get the chat model activations for the same tokens.
        chat_logits = chat_model(tokens)

    print(original_logits.shape, chat_logits.shape)

    # === Run the models to obtain cached activations at the chosen hook point ===
    _, cache_A = base_model.run_with_cache(tokens, names_filter=hook_point)
    _, cache_B = chat_model.run_with_cache(tokens, names_filter=hook_point)

    act_A = cache_A[hook_point]
    act_B = cache_B[hook_point]

    for pos in wait_positions:
        token_id = tokens[0, pos].item()
        logits_A = base_model.run_with_hooks(tokens[:, :pos], fwd_hooks=[(hook_point, make_hook_fn(act_A, act_B, pos, 0))])
        logits_B = chat_model.run_with_hooks(tokens[:, :pos], fwd_hooks=[(hook_point, make_hook_fn(act_A, act_B, pos, 1))])
        delta_base_logit = logits_A[0][-1][token_id] - original_logits[0][pos-1][token_id]
        delta_chat_logit = logits_B[0][-1][token_id] - chat_logits[0][pos-1][token_id]

        orig_base_logit_list.append(original_logits[0][pos-1][token_id].item())
        orig_chat_logit_list.append(chat_logits[0][pos-1][token_id].item())
        new_base_logit_list.append(logits_A[0][-1][token_id].item())
        new_chat_logit_list.append(logits_B[0][-1][token_id].item())

        total_wait_tokens += 1
        if total_wait_tokens >= 100:
            break
    if total_wait_tokens >= 100:
        break

final_result_dict = {
    "orig_base_logit": orig_base_logit_list,
    "orig_chat_logit": orig_chat_logit_list,
    "new_base_logit": new_base_logit_list,
    "new_chat_logit": new_chat_logit_list
}

import json
with open(f"../results/{save_name}_{target_name}_{"random" if random_ablate else "top"}_{"amp" if amplify else "no"}_logits.json", 'w') as f:
    json.dump(final_result_dict, f)
    


OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB. GPU 0 has a total capacity of 79.14 GiB of which 170.75 MiB is free. Including non-PyTorch memory, this process has 78.96 GiB memory in use. Of the allocated memory 77.28 GiB is allocated by PyTorch, and 1.18 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [19]:
import json
weight_path = "../checkpoints/version_0/qwen-1.5b_13.pt"
filename = "../results/" + weight_path[2:-3].replace("/","__") + ".json"
with open(filename[:-5] + "_feat_annotations.json", 'r') as file:
    data = json.load(file)
print(data['base'])

{'914': '```Category: math```\nThe feature is activated by mathematical expressions and problem-solving contexts, suggesting a strong focus on mathematical reasoning and calculations.', '17831': '```Category: math```  \nThe examples provided primarily contain mathematical expressions and equations, indicating that the feature is heavily engaged in a mathematical context.', '31389': '```Category: math```\nThe feature is activated by specific mathematical expressions and conditions, indicating a strong focus on numerical relationships and constraints often found in mathematical or algorithmic contexts.', '11877': '```Category: math```  \nThe feature is primarily activated in contexts discussing mathematical concepts or problems, as evidenced by the presence of mathematical expressions and terminology throughout the examples.', '25796': '```Category: math```  \nThe feature appears to be related to mathematical reasoning, as indicated by the presence of equations and mathematical terminolo

In [33]:
cnt = 0
first_feat_idx = None
for key in data['reasoning']:
    if 'Type: Deductive' in data['reasoning'][key]:
        cnt += 1
        if not first_feat_idx:
            first_feat_idx = key
print(first_feat_idx)
print(cnt/100)

23569
0.02


In [34]:
cnt = 0
first_feat_idx = None
for key in data['base']:
    if 'Type: Deductive' in data['base'][key]:
        cnt += 1
        if not first_feat_idx:
            first_feat_idx = key

print(first_feat_idx)
print(cnt/100)

27603
0.01
